## 利用信息抽取技术搭建知识库

在这个notebook文件中，有些模板代码已经提供给你，但你还需要实现更多的功能来完成这个项目。除非有明确要求，你无须修改任何已给出的代码。以**'【练习】'**开始的标题表示接下来的代码部分中有你需要实现的功能。这些部分都配有详细的指导，需要实现的部分也会在注释中以'TODO'标出。请仔细阅读所有的提示。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

---

### 让我们开始吧

本项目的目的是结合命名实体识别、依存语法分析、实体消歧、实体统一对网站开放语料抓取的数据建立小型知识图谱。

在现实世界中，你需要拼凑一系列的模型来完成不同的任务；举个例子，用来预测狗种类的算法会与预测人类的算法不同。在做项目的过程中，你可能会遇到不少失败的预测，因为并不存在完美的算法和模型。你最终提交的不完美的解决方案也一定会给你带来一个有趣的学习经验！


---


## 步骤 1：实体统一

实体统一做的是对同一实体具有多个名称的情况进行统一，将多种称谓统一到一个实体上，并体现在实体的属性中（可以给实体建立“别称”属性）

例如：对“河北银行股份有限公司”、“河北银行公司”和“河北银行”我们都可以认为是一个实体，我们就可以将通过提取前两个称谓的主要内容，得到“河北银行”这个实体关键信息。

公司名称有其特点，例如后缀可以省略、上市公司的地名可以省略等等。在data/dict目录中提供了几个词典，可供实体统一使用。
- company_suffix.txt是公司的通用后缀词典
- company_business_scope.txt是公司经营范围常用词典
- co_Province_Dim.txt是省份词典
- co_City_Dim.txt是城市词典
- stopwords.txt是可供参考的停用词

### 练习1：
编写main_extract函数，实现对实体的名称提取“主体名称”的功能。

In [110]:
import jieba
import jieba.posseg as pseg
import re
import datetime

dict_entity_name_unify = {}

# 从输入的“公司名”中提取主体
def main_extract(input_str, stop_word, d_4_delete, d_city_province):
    # 开始分词并处理
    seg = pseg.cut(input_str)
    seg_lst = remove_word(seg, stop_word, d_4_delete)
    seg_lst = city_prov_ahead(seg_lst, d_city_province)
    result = ''.join(seg_lst)
    if result != input_str:
        if result not in dict_entity_name_unify:
            dict_entity_name_unify[result] = ""
        dict_entity_name_unify[result] = dict_entity_name_unify[result] + "|" + input_str
    return result

    
#TODO：实现公司名称中地名提前
def city_prov_ahead(seg, d_city_province):
    city_prov_lst = []
    # TODO ...
    for word in seg:
        if word in d_city_province:
            city_prov_lst.append(word)
    seg_lst = [word for word in seg if word not in city_prov_lst]
    return city_prov_lst + seg_lst


#TODO：替换特殊符号
def remove_word(seg, stop_word, d_4_delete):
    # TODO ...
    seg_lst = [word for word, flag in seg if word not in stop_word and word not in d_4_delete]
    return seg_lst


# 初始化，加载词典
def my_initial():
    fr1 = open(r"./data/dict/co_City_Dim.txt", encoding='utf-8')
    fr2 = open(r"./data/dict/co_Province_Dim.txt", encoding='utf-8')
    fr3 = open(r"./data/dict/company_business_scope.txt", encoding='utf-8')
    fr4 = open(r"./data/dict/company_suffix.txt", encoding='utf-8')
    #城市名
    lines1 = fr1.readlines()
    d_4_delete = []
    d_city_province = [re.sub(r'(\r|\n)*','',line) for line in lines1]
    #省份名
    lines2 = fr2.readlines()
    l2_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines2]
    d_city_province.extend(l2_tmp)
    #公司后缀
    lines3 = fr3.readlines()
    l3_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines3]
    lines4 = fr4.readlines()
    l4_tmp = [re.sub(r'(\r|\n)*','',line) for line in lines4]
    d_4_delete.extend(l4_tmp)
    #get stop_word
    fr = open(r'./data/dict/stopwords.txt', encoding='utf-8')   
    stop_word = fr.readlines()
    stop_word_after = [re.sub(r'(\r|\n)*','',stop_word[i]) for i in range(len(stop_word))]
#     stop_word_after[-1] = stop_word[-1]
    stop_word = stop_word_after
    return d_4_delete, stop_word, d_city_province

In [2]:
# TODO：测试实体统一用例
d_4_delete, stop_word, d_city_province = my_initial()
company_name = "河北银行股份有限公司"
company_name = main_extract(company_name, stop_word, d_4_delete, d_city_province)
print(company_name)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.799 seconds.
Prefix dict has been built successfully.


河北银行


## 步骤 2：实体识别
有很多开源工具可以帮助我们对实体进行识别。常见的有LTP、StanfordNLP、FoolNLTK等等。

本次采用FoolNLTK实现实体识别，fool是一个基于bi-lstm+CRF算法开发的深度学习开源NLP工具，包括了分词、实体识别等功能，大家可以通过fool很好地体会深度学习在该任务上的优缺点。

在‘data/train_data.csv’和‘data/test_data.csv’中是从网络上爬虫得到的上市公司公告，数据样例如下：

In [3]:
import pandas as pd

train_data = pd.read_csv('./data/info_extract/train_data.csv', encoding = 'gb2312', header=0)
train_data.head()

,id,sentence,tag,member1,member2
0,6461,与本公司关系:受同一公司控制 2，杭州富生电器有限公司企业类型: 有限公司注册地址: 富阳市...,0,0,0
1,2111,三、关联交易标的基本情况 1、交易标的基本情况 公司名称:红豆集团财务有限公司 公司地址:无...,0,0,0
2,9603,2016年协鑫集成科技股份有限公司向瑞峰（张家港）光伏科技有限公司支付设备款人民币4，515...,1,协鑫集成科技股份有限公司,瑞峰（张家港）光伏科技有限公司
3,3456,证券代码:600777 证券简称:新潮实业 公告编号:2015-091 烟台新潮实业股份有限...,0,0,0
4,8844,本集团及广发证券股份有限公司持有辽宁成大股份有限公司股票的本期变动系买卖一揽子沪深300指数...,1,广发证券股份有限公司,辽宁成大股份有限公司


In [4]:
test_data = pd.read_csv('./data/info_extract/test_data.csv', encoding = 'gb2312', header=0)
test_data.head()

,id,sentence
0,9259,2015年1月26日，多氟多化工股份有限公司与李云峰先生签署了《附条件生效的股份认购合同》
1,9136,2、2016年2月5日，深圳市新纶科技股份有限公司与侯毅先
2,220,2015年10月26日，山东华鹏玻璃股份有限公司与张德华先生签署了附条件生效条件的《股份认购合同》
3,9041,2、2015年12月31日，印纪娱乐传媒股份有限公司与肖文革签订了《印纪娱乐传媒股份有限公司...
4,10041,一、金发科技拟与熊海涛女士签订《股份转让协议》，协议约定：以每股1.0509元的收购价格，收...


我们选取一部分样本进行标注，即train_data，该数据由5列组成。id列表示原始样本序号；sentence列为我们截取的一段关键信息；如果关键信息中存在两个实体之间有股权交易关系则tag列为1，否则为0；如果tag为1，则在member1和member2列会记录两个实体出现在sentence中的名称。

剩下的样本没有标注，即test_data，该数据只有id和sentence两列，希望你能训练模型对test_data中的实体进行识别，并判断实体对之间有没有股权交易关系。

### 练习2：
将每句句子中实体识别出，存入实体词典，并用特殊符号替换语句。


In [5]:
import fool
words, ners = fool.analysis('多氟多化工股份有限公司与李云峰先生签署了《附条件生效的股份认购合同》')
ners

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: 

Instructions for updating:
Use the retry module or similar alternatives.


[[(0, 11, 'company', '多氟多化工股份有限公司'), (12, 15, 'person', '李云峰')]]

In [111]:
# 处理test数据，利用开源工具进行实体识别和并使用实体统一函数存储实体
import fool
import pandas as pd
from copy import copy


test_data = pd.read_csv('./data/info_extract/test_data.csv', encoding = 'gb2312', header=0)
test_data['ner'] = None
ner_id = 1001
ner_dict_new = {}  # 存储所有实体
ner_dict_reverse_new = {}  # 存储所有实体

for i in range(len(test_data)):
    sentence = copy(test_data.iloc[i, 1])
    # TODO：调用fool进行实体识别，得到words和ners结果
    words, ners = fool.analysis(sentence)
    
    ners[0].sort(key=lambda x:x[0], reverse=True)
    for start, end, ner_type, ner_name in ners[0]:
        if ner_type=='company' or ner_type=='person':
            # TODO：调用实体统一函数，存储统一后的实体
            # 并自增ner_id
            company_main_name = main_extract(ner_name, stop_word, d_4_delete, d_city_province)
            if company_main_name not in ner_dict_new:
                # ner_id 从 1001开始
                ner_dict_new[company_main_name] = ner_id
                ner_id += 1
            # 在句子中用编号替换实体名
            sentence = sentence[:start] + ' ner_' + str(ner_dict_new[company_main_name]) + '_ ' + sentence[end:]
    test_data.iloc[i, -1] = sentence

X_test = test_data[['ner']]

In [28]:
X_test

,ner
0,2015年1月26日， ner_1002_ 与 ner_1001_ 先生签署了《附条件生效的...
1,2、2016年2月5日， ner_1004_ 与 ner_1003_ 先
2,2015年10月26日， ner_1006_ 与 ner_1005_ 先生签署了附条件生效条...
3,2、2015年12月31日， ner_1008_ 与 ner_1007_ 签订了《印纪娱乐传...
4,一、 ner_1010_ 拟与 ner_1009_ 女士签订《股份转让协议》，协议约定：以每...
...,...
414,近日，该子公司已完成工商注册登记手续，并领取了南京市工商行政管理局颁发的<企业法人营业执照>...
415,(二)本次交易构成关联交易正元投资拟认购金额不低于 13 亿元且不低于本次配套融资总额的 2...
416,证券代码:600225 证券简称: ner_1642_ 公告编号:临 2015-118 n...
417,2015年3月31日， ner_1644_ 与 ner_1643_ 签署了附条件生效的《湖南...


In [112]:
# 处理train数据，利用开源工具进行实体识别和并使用实体统一函数存储实体
train_data = pd.read_csv('./data/info_extract/train_data.csv', encoding = 'gb2312', header=0)
train_data['ner'] = None

for i in range(len(train_data)):
    # 判断正负样本
    if train_data.iloc[i,:]['member1']=='0' and train_data.iloc[i,:]['member2']=='0':
        sentence = copy(train_data.iloc[i, 1])
        # TODO：调用fool进行实体识别，得到words和ners结果
        words, ners = fool.analysis(sentence)
    
        ners[0].sort(key=lambda x:x[0], reverse=True)
        for start, end, ner_type, ner_name in ners[0]:
            if ner_type=='company' or ner_type=='person':
                # TODO：调用实体统一函数，存储统一后的实体
                # 并自增ner_id
                company_main_name = main_extract(ner_name, stop_word, d_4_delete, d_city_province)
                if company_main_name not in ner_dict_new:
                    ner_dict_new[company_main_name] = ner_id
                    ner_id += 1
                # 在句子中用编号替换实体名
                sentence = sentence[:start] + ' ner_' + str(ner_dict_new[company_main_name]) + '_ ' + sentence[end:]
        train_data.iloc[i, -1] = sentence
    else:
        # 将训练集中正样本已经标注的实体也使用编码替换
        sentence = copy(train_data.iloc[i,:]['sentence'])
        for company_main_name in [train_data.iloc[i,:]['member1'], train_data.iloc[i,:]['member2']]:
            # TODO：调用实体统一函数，存储统一后的实体
            # 并自增ner_id
            company_main_name_new = main_extract(company_main_name, stop_word, d_4_delete, d_city_province)
            if company_main_name_new not in ner_dict_new:
                ner_dict_new[company_main_name_new] = ner_id
                ner_id += 1
            # 在句子中用编号替换实体名
            sentence = re.sub(company_main_name, ' ner_%s_ '%(str(ner_dict_new[company_main_name_new])), sentence)
        train_data.iloc[i, -1] = sentence
        
ner_dict_reverse_new = {id:name for name, id in ner_dict_new.items()}        
y = train_data.loc[:,['tag']]
train_num = len(train_data)
X_train = train_data[['ner']]

# 将train和test放在一起提取特征
X = pd.concat([X_train, X_test], axis=0)

In [49]:
len(X_train), len(X_test), len(X)

(850, 419, 1269)

In [68]:
X.iloc[0].tolist()

['与本公司关系:受同一公司控制 2， ner_1646_ 企业类型: 有限公司注册地址: 富阳市东洲街道东洲工业功能区九号路 1 号 法定代表人: ner_1645_ 注册资本: ?16，000 万元经营范围: 许可经营项目:制造高效节能感应电机;普通货运。']

## 步骤 3：关系抽取


目标：借助句法分析工具，和实体识别的结果，以及文本特征，基于训练数据抽取关系，并存储进图数据库。

本次要求抽取股权交易关系，关系为无向边，不要求判断投资方和被投资方，只要求得到双方是否存在交易关系。

模板建立可以使用“正则表达式”、“实体间距离”、“实体上下文”、“依存句法”等。

答案提交在submit目录中，命名为info_extract_submit.csv和info_extract_entity.csv。
- info_extract_entity.csv格式为：第一列是实体编号，第二列是实体名（实体统一的多个实体名用“|”分隔）
- info_extract_submit.csv格式为：第一列是关系中实体1的编号，第二列为关系中实体2的编号。

示例：
- info_extract_entity.csv

| 实体编号 | 实体名 |
| ------ | ------ |
| 1001 | 小王 |
| 1002 | A化工厂 |

- info_extract_submit.csv

| 实体1 | 实体2 |
| ------ | ------ |
| 1001 | 1003 |
| 1002 | 1001 |

### 练习3：提取文本tf-idf特征

去除停用词，并转换成tfidf向量。

In [69]:
# code
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from pyltp import Segmentor


# 实体符号加入分词词典
with open('./data/user_dict.txt', 'w') as fw:
    for v in ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十']:
        fw.write( v + '号企业 ni\n')

# 初始化实例
segmentor = Segmentor()  
# 加载模型，加载自定义词典
segmentor.load_with_lexicon('./ltp_data_v3.4.0/cws.model', './data/user_dict.txt')  

# 加载停用词
fr = open(r'./data/dict/stopwords.txt', encoding='utf-8')   
stop_word = fr.readlines()
stop_word = [re.sub(r'(\r|\n)*', '', stop_word[i]) for i in range(len(stop_word))]

# 分词
# f = lambda x: ' '.join([word for word in segmentor.segment(re.sub(r'ner\_\d\d\d\d\_','',x)) if word not in stop_word])
f = lambda x: ' '.join([word for word in segmentor.segment(x) if word not in stop_word and not re.findall(r'ner\_\d\d\d\d\_', word)])
corpus = X['ner'].map(f).tolist()


from sklearn.feature_extraction.text import TfidfVectorizer
# TODO：提取tfidf特征
vectorizer = TfidfVectorizer()  # 定一个tf-idf的vectorizer
X_tfidf = vectorizer.fit_transform(corpus).toarray()  # 结果存放在X矩阵
print(X_tfidf)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.26970885 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [73]:
X_tfidf.shape

(1269, 6078)

### 练习4：提取句法特征
除了词语层面的句向量特征，我们还可以从句法入手，提取一些句法分析的特征。

参考特征：

1、企业实体间距离

2、企业实体间句法距离

3、企业实体分别和关键触发词的距离

4、实体的依存关系类别

In [93]:
s = '我喜欢你'
words = segmentor.segment(s)
tags = postagger.postag(words)
parser = Parser() # 初始化实例
parser.load('./ltp_data_v3.4.0/parser.model')  # 加载模型
arcs = parser.parse(words, tags)  # 句法分析
arcs_lst = list(map(list, zip(*[[arc.head, arc.relation] for arc in arcs])))
print(arcs_lst)
# 实体的依存关系类别
rely_id = [arc.head for arc in arcs]  # 提取依存父节点id
relation = [arc.relation for arc in arcs]  # 提取依存关系
heads = ['Root' if id == 0 else words[id - 1] for id in rely_id]  # 匹配依存父节点词语
for i in range(len(words)):
    print(relation[i] + '(' + words[i] + ', ' + heads[i] + ')')

[[2, 0, 2], ['SBV', 'HED', 'VOB']]
SBV(我, 喜欢)
HED(喜欢, Root)
VOB(你, 喜欢)


In [99]:
s = '我喜欢你'
words = segmentor.segment(s)
tags = postagger.postag(words)
parser = Parser() # 初始化实例
parser.load('./ltp_data_v3.4.0/parser.model')  # 加载模型
arcs = parser.parse(words, tags)  # 句法分析
arcs_lst = list(map(list, zip(*[[arc.head, arc.relation] for arc in arcs])))

# 句法分析结果输出
parse_result = pd.DataFrame([[a,b,c,d] for a,b,c,d in zip(list(words), list(tags), arcs_lst[0], arcs_lst[1])], index=range(1, len(words)+1))
parser.release()  # 释放模型
parse_result

,0,1,2,3
1,我,r,2,SBV
2,喜欢,v,0,HED
3,你,r,2,VOB


In [101]:
# -*- coding: utf-8 -*-
from pyltp import Parser
from pyltp import Segmentor
from pyltp import Postagger
import networkx as nx
import pylab
import re
import numpy as np


postagger = Postagger() # 初始化实例
postagger.load_with_lexicon('./ltp_data_v3.4.0/pos.model', './data/user_dict.txt')  # 加载模型
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon('./ltp_data_v3.4.0/cws.model', './data/user_dict.txt')  # 加载模型
SEN_TAGS = ["SBV","VOB","IOB","FOB","DBL","ATT","ADV","CMP","COO","POB","LAD","RAD","IS","HED"]

def parse(s, isGraph = False):
    """
    对语句进行句法分析，并返回句法结果
    """
    tmp_ner_dict = {}
    num_lst = ['一', '二', '三', '四', '五', '六', '七', '八', '九', '十']

    # 将公司代码替换为特殊称谓，保证分词词性正确
    for i, ner in enumerate(list(set(re.findall(r'(ner\_\d\d\d\d\_)', s)))):
        try:
            tmp_ner_dict[num_lst[i]+'号企业'] = ner
        except IndexError:
            # TODO：定义错误情况的输出
            num_lst.append(str(i))
            tmp_ner_dict[num_lst[i] + '号企业'] = ner
        s = s.replace(ner, num_lst[i]+'号企业')
        
    words = segmentor.segment(s)
    tags = postagger.postag(words)
    parser = Parser() # 初始化实例
    parser.load('./ltp_data_v3.4.0/parser.model')  # 加载模型
    arcs = parser.parse(words, tags)  # 句法分析
    arcs_lst = list(map(list, zip(*[[arc.head, arc.relation] for arc in arcs])))
    
    # 句法分析结果输出
    parse_result = pd.DataFrame([[a,b,c,d] for a,b,c,d in zip(list(words), list(tags), arcs_lst[0], arcs_lst[1])], index=range(1, len(words)+1))
    parser.release()  # 释放模型
    # TODO：提取企业实体依存句法类型
    result = []
    
    # 实体的依存关系类别
    rely_id = [arc.head for arc in arcs]  # 提取依存父节点id
    relation = [arc.relation for arc in arcs]  # 提取依存关系
    heads = ['Root' if id == 0 else words[id - 1] for id in rely_id]  # 匹配依存父节点词语

    company_list = list(tmp_ner_dict.keys())
    str_enti_1 = "一号企业"
    str_enti_2 = "二号企业"
    l_w = list(words)
    is_two_company = str_enti_1 in l_w and str_enti_2 in l_w
    if is_two_company:
        second_entity_index = l_w.index(str_enti_2)
        entity_sentence_type = parse_result.iloc[second_entity_index, -1]
        if entity_sentence_type in SEN_TAGS:
            result.append(SEN_TAGS.index(entity_sentence_type))
        else:
            result.append(-1)
    else:
        result.append(-1)

    if isGraph:
        g = Digraph('测试图片')
        g.node(name='Root')
        for word in words:
            g.node(name=word, fontname="SimHei")
        for i in range(len(words)):
            if relation[i] not in ['HED']:
                g.edge(words[i], heads[i], label=relation[i], fontname="SimHei")
            else:
                if heads[i] == 'Root':
                    g.edge(words[i], 'Root', label=relation[i], fontname="SimHei")
                else:
                    g.edge(heads[i], 'Root', label=relation[i], fontname="SimHei")
        g.view()
        
    # 企业实体间句法距离
    distance_e_jufa = 0
    if is_two_company:
        distance_e_jufa = shortest_path(parse_result, list(words), str_enti_1, str_enti_2, isGraph=False)
    result.append(distance_e_jufa)

    # 企业实体间距离
    distance_entity = 0
    if is_two_company:
        distance_entity = np.abs(l_w.index(str_enti_1) - l_w.index(str_enti_2))
    result.append(distance_entity)

    # 投资关系关键词
    key_words = ["收购","竞拍","转让","扩张","并购","注资","整合","并入","竞购","竞买","支付","收购价","收购价格","承购","购得","购进",
             "购入","买进","买入","赎买","购销","议购","函购","函售","抛售","售卖","销售","转售"]
    # TODO：*根据关键词和对应句法关系提取特征（如没有思路可以不完成）
    k_w = None
    for w in words:
        if w in key_words:
            k_w = w
            break
    dis_key_e_1 = -1
    dis_key_e_2 = -1
    if k_w != None and is_two_company:
        k_w = str(k_w)
        l_w = list(words)
        dis_key_e_1 = np.abs(l_w.index(str_enti_1) - l_w.index(k_w))
        dis_key_e_2 = np.abs(l_w.index(str_enti_2) - l_w.index(k_w))
    result.append(dis_key_e_1)
    result.append(dis_key_e_2)

    return result


def shortest_path(arcs_ret, words, source, target, isGraph = False):
    """
    求出两个词最短依存句法路径，不存在路径返回-1
    arcs_ret：句法分析结果
    source：实体1
    target：实体2
    """
    G = nx.DiGraph()
    # 为这个网络添加节点...
    for i in list(arcs_ret.index):
        G.add_node(i)
    # TODO：在网络中添加带权中的边...（注意，我们需要的是无向边）
    for i in range(len(arcs_ret)):
        head = arcs_ret.iloc[i, -2]
        index = i + 1 # 从1开始
        G.add_edge(index, head)

    if isGraph:
        nx.draw(G, with_labels=True)
#         plt.savefig("undirected_graph_2.png")
        plt.close()

    try:
        # TODO：利用nx包中shortest_path_length方法实现最短距离提取
        source_index = words.index(source) + 1 #从1开始
        target_index = words.index(target) + 1 #从1开始
        distance = nx.shortest_path_length(G, source=source_index, target=target_index)
        # print("'%s'与'%s'在依存句法分析图中的最短距离为:  %s" % (source, target, distance))
        return distance
    except:
        return -1


def get_feature(s):
    """
    汇总上述函数汇总句法分析特征与TFIDF特征
    """
    # TODO：汇总上述函数汇总句法分析特征与TFIDF特征
    sen_feature = []
    len_s = len(s)
    for i in range(len_s):
        f_e = parse(s[i], isGraph = False)
        sen_feature.append(f_e)
    sen_feature = np.array(sen_feature)
    features = np.concatenate((X_tfidf, sen_feature), axis=1)
    return features

In [117]:
import os
f_v_s_path = "./data/feature_vector.npy"
is_exist_f_v = os.path.exists(f_v_s_path)
corpus_1 = X['ner'].tolist()
len_train_data = len(train_data)
features = []
if not is_exist_f_v:
    features = get_feature(corpus_1)
    np.save(f_v_s_path, features)
else:
    features = np.load(f_v_s_path)
features_train = features[:len_train_data, :]

In [107]:
features_train.shape

(850, 6083)

### 练习5：建立分类器

利用已经提取好的tfidf特征以及parse特征，建立分类器进行分类任务。

In [114]:
# 建立分类器进行分类
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.naive_bayes import BernoulliNB

seed = 2019

y = train_data.loc[:, ['tag']]
y = np.array(y.values)
y = y.reshape(-1)
Xtrain, Xtest, ytrain, ytest = train_test_split(features_train,  y, test_size=0.2, random_state=seed)

def logistic_class(Xtrain, Xtest, ytrain, ytest):
    cross_validator = KFold(n_splits=5, shuffle=True, random_state=seed)
    lr = LogisticRegression(penalty = "l1", solver='liblinear')
    # params = {"penalty":["l1","l2"], "C":[0.1,1.0,10.0,20.0,30.0,100.0]}
    params = {"C":[0.1,1.0,10.0,15.0,20.0,30.0,40.0,50.0]}
    grid = GridSearchCV(estimator=lr, param_grid=params, cv=cross_validator)
    grid.fit(Xtrain, ytrain)
    print("最优参数为：",grid.best_params_)
    model = grid.best_estimator_
    y_pred = model.predict(Xtest)
    y_test = [str(value) for value in ytest]
    y_pred = [str(value) for value in y_pred]
    # train_score = model.score(X_train, y_train)
    # print("train_score", train_score)
    # test_score = model.score(X_test, y_test)
    # print("test_score", test_score)
    # f1_score_value = f1_score(y_test, y_pred)
    # print("F1-Score: {}".format(f1_score_value))
    proba_value = model.predict_proba(Xtest)
    p = proba_value[:, 1]
    print("Logistic=========== ROC-AUC score: %.3f" % roc_auc_score(y_test, p))
    report = classification_report(y_pred=y_pred,y_true=y_test)
    print(report)
    return model

# TODO：保存Test_data分类结果
# 答案提交在submit目录中，命名为info_extract_submit.csv和info_extract_entity.csv。
# info_extract_entity.csv格式为：第一列是实体编号，第二列是实体名（实体统一的多个实体名用“|”分隔）
# info_extract_submit.csv格式为：第一列是关系中实体1的编号，第二列为关系中实体2的编号。
s_model = logistic_class(Xtrain, Xtest, ytrain, ytest)
features_test = features[len_train_data:, :]
y_pred_test = s_model.predict(features_test)
l_X_test_ner = X_test.values.tolist()
entity_dict = {}
relation_list = []

for i, label in enumerate(y_pred_test):
    if label == 1:
        cur_ner_content = str(l_X_test_ner[i])
        ner_list = list(set(re.findall(r'(ner\_\d\d\d\d\_)', cur_ner_content)))
        if len(ner_list) == 2:
            r_e_l = []
            for i, ner in enumerate(ner_list):
                split_list = str.split(ner, "_")
                if len(split_list) == 3:
                    ner_id = int(split_list[1])
                    if ner_id in ner_dict_reverse_new:
                        if ner_id not in entity_dict:
                            company_main_name = ner_dict_reverse_new[ner_id]
                            if company_main_name in dict_entity_name_unify:
                                entity_dict[ner_id] = company_main_name + dict_entity_name_unify[company_main_name]
                            else:
                                entity_dict[ner_id] = company_main_name
                        r_e_l.append(ner_id)
            if len(r_e_l) == 2:
                relation_list.append(r_e_l)

entity_list = [[item[0], item[1]] for item in entity_dict.items()]
pd_enti = pd.DataFrame(np.array(entity_list), columns=['实体编号','实体名'])
pd_enti.to_csv("./data/info_extract_entity.csv",index=0, encoding='utf_8_sig')
pd_re = pd.DataFrame(np.array(relation_list), columns=['实体1','实体2'])
pd_re.to_csv("./data/info_extract_submit.csv",index=0,encoding='utf_8_sig')

最优参数为： {'C': 10.0}
Logistic=========== ROC-AUC score: 0.934
              precision    recall  f1-score   support

           0       0.91      0.99      0.94       148
           1       0.78      0.32      0.45        22

    accuracy                           0.90       170
   macro avg       0.84      0.65      0.70       170
weighted avg       0.89      0.90      0.88       170



In [115]:
entity = pd.read_csv('./data/info_extract_entity.csv', encoding='utf_8_sig', header=0)
entity.head()

,实体编号,实体名
0,1002,氟化工|多氟多化工股份有限公司|多氟多化工股份有限公司
1,1001,李云峰
2,1003,侯毅
3,1004,深圳市新纶科技股份|深圳市新纶科技股份有限公司
4,1006,山东华鹏玻璃|山东华鹏玻璃股份有限公司|山东华鹏玻璃股份有限公司|山东华鹏玻璃股份有限公司|...


In [116]:
relation = pd.read_csv('./data/info_extract_submit.csv', encoding='utf_8_sig', header=0)
relation.head()

,实体1,实体2
0,1002,1001
1,1003,1004
2,1006,1005
3,1008,1007
4,1010,1009


### 练习6：操作图数据库
对关系最好的描述就是用图，那这里就需要使用图数据库，目前最常用的图数据库是noe4j，通过cypher语句就可以操作图数据库的增删改查。可以参考“https://cuiqingcai.com/4778.html”。

本次作业我们使用neo4j作为图数据库，neo4j需要java环境，请先配置好环境。

将我们提出的实体关系插入图数据库，并查询某节点的3层投资关系，即三个节点组成的路径（如果有的话）。如果无法找到3层投资关系，请查询出任意指定节点的投资路径。

In [127]:
from py2neo import Node, Relationship, Graph

graph = Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="666666"
)

for v in relation_list:
    a = Node('Company', name=v[0])
    b = Node('Company', name=v[1])
    
    # 本次不区分投资方和被投资方，无向图
    r = Relationship(a, 'INVEST', b)
    s = a | b | r
    graph.create(s)
    r = Relationship(b, 'INVEST', a)
    s = a | b | r
    graph.create(s)

In [128]:
# TODO：查询某节点的3层投资关系
import random

result_2 = []
result_3 = []
for value in entity_list:
    ner_id = value[0]
    str_sql_3 = "match data=(na:Company{{name:'{0}'}})-[:INVEST]->(nb:Company)-[:INVEST]->(nc:Company) where na.name <> nc.name return data".format(str(ner_id))
    result_3 = graph.run(str_sql_3).data()
    if len(result_3) > 0:
        break
if len(result_3) > 0:
    print("step1")
    print(result_3)
else:
    print("step2")
    random_index = random.randint(0, len(entity_list) - 1)
    random_ner_id = entity_list[random_index][0]
    str_sql_2 = "match data=(na:Company{{name:'{0}'}})-[*2]->(nb:Company) return data".format(str(random_ner_id))
    result_2 = graph.run(str_sql_2).data()
    print(result_2)

step2
[]


## 步骤4：实体消歧
解决了实体识别和关系的提取，我们已经完成了一大截，但是我们提取的实体究竟对应知识库中哪个实体呢？下图中，光是“苹果”就对应了13个同名实体。
<img src="../image/baike2.png", width=340, heigth=480>

在这个问题上，实体消歧旨在解决文本中广泛存在的名称歧义问题，将句中识别的实体与知识库中实体进行匹配，解决实体歧义问题。


### 练习7：
匹配test_data.csv中前25条样本中的人物实体对应的百度百科URL（此部分样本中所有人名均可在百度百科中链接到）。

利用scrapy、beautifulsoup、request等python包对百度百科进行爬虫，判断是否具有一词多义的情况，如果有的话，选择最佳实体进行匹配。

使用URL为‘https://baike.baidu.com/item/’+人名 可以访问百度百科该人名的词条，此处需要根据爬取到的网页识别该词条是否对应多个实体，如下图：
<img src="../image/baike1.png", width=440, heigth=480>
如果该词条有对应多个实体，请返回正确匹配的实体URL，例如该示例网页中的‘https://baike.baidu.com/item/陆永/20793929’。

- 提交文件：entity_disambiguation_submit.csv
- 提交格式：第一列为实体id（与info_extract_submit.csv中id保持一致），第二列为对应URL。
- 示例：

| 实体编号 | URL |
| ------ | ------ |
| 1001 | https://baike.baidu.com/item/陆永/20793929 |
| 1002 | https://baike.baidu.com/item/王芳/567232 |


In [129]:
import jieba
import pandas as pd

# 找出test_data.csv中前25条样本所有的人物名称，以及人物所在文档的上下文内容
test_data = pd.read_csv('./data/info_extract/test_data.csv', encoding = 'gb2312', header=0)

# 存储人物以及上下文信息（key为人物ID，value为人物名称、人物上下文内容）
list_person_content = {}

# 观察上下文的窗口大小
window = 20

f = lambda x: ' '.join([word for word in segmentor.segment(x)])
corpus= test_data['sentence'].map(f).tolist()
vectorizer = TfidfVectorizer()  # 定一个tf-idf的vectorizer
X_tfidf = vectorizer.fit_transform(corpus).toarray()  # 结果存放在X矩阵

# 遍历前25条样本
for i in range(25):
    sentence = str(copy(test_data.iloc[i, 1]))
    len_sen = len(sentence)
    words, ners = fool.analysis(sentence)
    ners[0].sort(key=lambda x: x[0], reverse=True)
    for start, end, ner_type, ner_name in ners[0]:
        if ner_type == 'person':
            # TODO：提取实体的上下文
            start_index = max(0, start - window)
            end_index = min(len_sen - 1, end - 1 + window)
            left_str = sentence[start_index:start]
            right_str = sentence[end - 1:end_index]
            left_str = ' '.join([word for word in segmentor.segment(left_str)])
            right_str = ' '.join([word for word in segmentor.segment(right_str)])
            new_str = left_str + " " +right_str

            content_vec = vectorizer.transform([new_str])

            ner_id = ner_dict_new[ner_name]
            if ner_id not in list_person_content:
                list_person_content[ner_id] = content_vec

In [133]:
# 利用爬虫得到每个人物名称对应的URL
# TODO：找到每个人物实体的词条内容。
from requests_html import HTMLSession
from requests_html import HTML
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import jieba


list_company_names = [company for value in entity_list for company in str.split(value[1], "|")]

list_person_url = []
url_prefix = "https://baike.baidu.com/item/"
url_error = "https://baike.baidu.com/error.html"

l_p_items = list(list_person_content.items())
len_items = len(l_p_items)

def get_para_vector(para_elems):
    str_res = ""
    for p_e in para_elems:
        str_res += re.sub(r'(\r|\n)*', '', p_e.text)
    str_res = ' '.join([word for word in jieba.cut(str_res)])
    content_vec = vectorizer.transform([str_res])
    content_vec = content_vec.toarray()[0]
    return content_vec

for index in range(len_items):
    value = l_p_items[index]

    person_id = value[0]
    vector_entity = csr_matrix(value[1])

    person_name = ner_dict_reverse_new[person_id]

    session = HTMLSession()
    url = url_prefix + person_name
    response = session.get(url)

    url_list = []
    if response.url != url_error:
        para_elems = response.html.find('.para')
        content_vec = get_para_vector(para_elems)
        url_list.append([response.url, content_vec])

        banks = response.html.find('.polysemantList-wrapper')

        if len(banks) > 0:
            banks_child = banks[0]
            persion_links = list(banks_child.absolute_links)
            for link in persion_links:
                r_link = session.get(link)

                if r_link.url == url_error:
                    continue

                para_elems = r_link.html.find('.para')
                content_vec = get_para_vector(para_elems)
                url_list.append([r_link.url, content_vec])

        vectorizer_list = [item[1] for item in url_list]
        vectorizer_list = csr_matrix(vectorizer_list)
        result = list(cosine_similarity(value[1], vectorizer_list)[0])
        max_index = result.index(max(result))
        list_person_url.append([person_id, person_name, url_list[max_index][0]])

print(list_person_url)

pd_re = pd.DataFrame(np.array(list_person_url), columns=['实体编号','名字','url'])
pd_re.to_csv("./data/entity_disambiguation_submit.csv",index=0,encoding='utf_8_sig')

[[1001, '李云峰', 'https://baike.baidu.com/item/%E6%9D%8E%E4%BA%91%E5%B3%B0/22102428#viewPageContent'], [1003, '侯毅', 'https://baike.baidu.com/item/%E4%BE%AF%E6%AF%85/12795458#viewPageContent'], [1005, '张德华', 'https://baike.baidu.com/item/%E5%BC%A0%E5%BE%B7%E5%8D%8E/7002694#viewPageContent'], [1007, '肖文革', 'https://baike.baidu.com/item/%E8%82%96%E6%96%87%E9%9D%A9/22791038#viewPageContent'], [1009, '熊海涛', 'https://baike.baidu.com/item/%E7%86%8A%E6%B5%B7%E6%B6%9B/10849366'], [1011, '宋琳', 'https://baike.baidu.com/item/%E5%AE%8B%E7%90%B3/16173836#viewPageContent'], [1014, '王友林', 'https://baike.baidu.com/item/%E7%8E%8B%E5%8F%8B%E6%9E%97/71412'], [1016, '彭聪', 'https://baike.baidu.com/item/%E5%BD%AD%E8%81%AA/19890127'], [1017, '曹飞', 'https://baike.baidu.com/item/%E6%9B%B9%E9%A3%9E/16542190#viewPageContent'], [1019, '颜军', 'https://baike.baidu.com/item/%E9%A2%9C%E5%86%9B/3476040'], [1021, '宋睿', 'https://baike.baidu.com/item/%E5%AE%8B%E7%9D%BF/2629451'], [1025, '邓冠华', 'https://baike.baidu.com/item/%